In [242]:
from gutenberg.cleanup import strip_headers
import requests
# http://nbviewer.jupyter.org/github/rare-technologies/gensim/blob/develop/docs/notebooks/atmodel_tutorial.ipynb

def load_etext(num):
  url ='http://www.gutenberg.org/files/{0}/{0}-0.txt'.format(num)
  data= requests.get(url)
  return data.text

texts =[]
text_nums=[1342, 158, 121, 105, 141,
           76,1837, 86, 3176, 8525,
           11,12, 29042, 620,35535,
           2701,21816, 34970, 12384,2694,
           98,19337, 786, 1023, 1400,
           2591]
text_names =['Pride and Prejudice', 'Emma', 'Northanger Abbey', "Persuasion", "Mansfield Park",
             'Adventures of Huckleberry Finn', 'The Prince and the Pauper', "A Connecticut Yankee in King Arthur's Court", "The Innocents Abroad", "Eve's Diary", 
             "Alice in Wonderland","Through the Looking-Glass", "A Tangled Tale", "Sylvie and Bruno", "Feeding the Mind",
            "Moby Dick", "The Confidence-Man: His Masquerade", "Pierre; or The Ambiguities", "Battle-Pieces and Aspects of the War","I and My Chimney",
             "A Tale of Two Cities","A Christmas Carol","Hard Times", "Bleak House", "Great Expectations",
             "Grimms’ Fairy Tales", 
            ]
for text_num in text_nums:
    text = strip_headers(load_etext(text_num)).strip()
    texts.append(text)

# Preprocess Text

- Lowercase
- Strip Tags
- Strip Punctuation
- Strip Multiple Whitespaces
- Strip Numeric
- Remove Stopwords
- Strip Shortwords
- Stem Text


In [243]:
from gensim.parsing.preprocessing import preprocess_string,preprocess_documents
from gensim import corpora,models, similarities


In [244]:
processed_docs = preprocess_documents(texts)

In [245]:
dictionary = corpora.Dictionary(processed_docs)
dictionary.save('gutenberg.dict')

In [246]:
print(dictionary)

Dictionary(27623 unique tokens: [u'fawn', u'\u2014\u2018the', u'woodi', u'spideri', u'\u201cdepth\u201d']...)


In [247]:
corpus = [dictionary.doc2bow(text) for text in processed_docs]
corpora.MmCorpus.serialize('gutenberg.mm', corpus) 

# Model

In [248]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=500)
lsi.save("gutenberg.model")
index = similarities.MatrixSimilarity(lsi[corpus])
# similarities.Similarity
index.save('gutenberg.index')

In [249]:
#lsi.show_topics()

In [250]:
rp = models.RpModel(corpus, id2word=dictionary, num_topics=500)
rp.save('rpgutenberg.model')
indexRP = similarities.MatrixSimilarity(rp[corpus])
indexRP.save('rpgutenberg.index')

In [251]:
def queryRP(doc):
    vec_bow = dictionary.doc2bow(preprocess_string(doc))
    vec_ = rp[vec_bow] 
    return vec_

In [252]:
#Wikipedia article about Polar Express plot
query_texts = ["""
A young boy, who used to adore Christmas, hears a train whistle roar. To his astonishment, he finds the train is waiting for him. He sees a conductor who then proceeds to look up at his window. He runs downstairs and goes outside. The conductor explains the train is called the Polar Express, and is journeying to the North Pole. The boy then boards the train, which is filled with chocolate and candy, as well as many other children in their pajamas.

As the train reaches the North Pole, the boy and the other children see thousands of Christmas elves gathered at the center of town waiting to send Santa Claus on his way. The boy is handpicked by Santa to receive the first gift of Christmas. Realizing that he could choose anything in the world, the boy asks for one bell from one of the reindeer's harnesses. The boy places the bell in the pocket of his robe and all the children watch as Santa takes off into the night for his annual deliveries.

Later, on the train ride home, the boy discovers that the bell has fallen through a hole in his pocket. The boy arrives home and goes to his bedroom as the train pulls away. On Christmas morning, his sister finds a small package for the boy under the tree, behind all of the other gifts. The boy opens the box and discovers that it is the bell, delivered by Santa who found it on the seat of his sleigh. When the boy rings the bell, both he and his sister marvel at the beautiful sound. His parents, however, are unable to hear the bell and remark that it must be broken. The book ends with the following line:

At one time, most of my friends could hear the bell, but as years passed, it fell silent for all of them. Even Sarah found one Christmas that she could no longer hear its sweet sound. Though I've grown old, the bell still rings for me, as it does for all who truly believe.
"""""]
query_RP = map(queryRP, query_texts)

In [253]:
simsRP = indexRP[query_RP[0]]
sorted(zip(text_names,simsRP),key=lambda x: x[1],reverse=True)

[('Northanger Abbey', 0.17584464),
 ('Great Expectations', 0.17337206),
 ('The Innocents Abroad', 0.16868544),
 ('A Tale of Two Cities', 0.15150893),
 ('The Prince and the Pauper', 0.14885591),
 ('Hard Times', 0.14732599),
 ('Grimms\xe2\x80\x99 Fairy Tales', 0.14298059),
 ("A Connecticut Yankee in King Arthur's Court", 0.14169708),
 ('Sylvie and Bruno', 0.12176942),
 ('Pride and Prejudice', 0.10697163),
 ('Moby Dick', 0.10662378),
 ('Through the Looking-Glass', 0.072553635),
 ('Alice in Wonderland', 0.072551966),
 ('I and My Chimney', 0.068820924),
 ('Emma', 0.060252212),
 ('Mansfield Park', 0.053078458),
 ("Eve's Diary", 0.051006593),
 ('Persuasion', 0.031630095),
 ('A Tangled Tale', 0.031630095),
 ('Feeding the Mind', 0.031630095),
 ('The Confidence-Man: His Masquerade', 0.031630095),
 ('Pierre; or The Ambiguities', 0.031630095),
 ('Battle-Pieces and Aspects of the War', 0.031630095),
 ('A Christmas Carol', 0.031630095),
 ('Bleak House', 0.031630095),
 ('Adventures of Huckleberry Fin

So far RP model seems to be giving the worst results due to the random variance it's adding into each book.

In [254]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=500)
lda.save('ldagutenberg.model')
indexLda = similarities.MatrixSimilarity(lda[corpus])
indexLda.save('ldagutenberg.index')

In [255]:
def queryLDA(doc):
    vec_bow = dictionary.doc2bow(preprocess_string(doc))
    vec_lda = lda[vec_bow] 
    return vec_lda

In [256]:
query_texts = ["""
A young boy, who used to adore Christmas, hears a train whistle roar. To his astonishment, he finds the train is waiting for him. He sees a conductor who then proceeds to look up at his window. He runs downstairs and goes outside. The conductor explains the train is called the Polar Express, and is journeying to the North Pole. The boy then boards the train, which is filled with chocolate and candy, as well as many other children in their pajamas.

As the train reaches the North Pole, the boy and the other children see thousands of Christmas elves gathered at the center of town waiting to send Santa Claus on his way. The boy is handpicked by Santa to receive the first gift of Christmas. Realizing that he could choose anything in the world, the boy asks for one bell from one of the reindeer's harnesses. The boy places the bell in the pocket of his robe and all the children watch as Santa takes off into the night for his annual deliveries.

Later, on the train ride home, the boy discovers that the bell has fallen through a hole in his pocket. The boy arrives home and goes to his bedroom as the train pulls away. On Christmas morning, his sister finds a small package for the boy under the tree, behind all of the other gifts. The boy opens the box and discovers that it is the bell, delivered by Santa who found it on the seat of his sleigh. When the boy rings the bell, both he and his sister marvel at the beautiful sound. His parents, however, are unable to hear the bell and remark that it must be broken. The book ends with the following line:

At one time, most of my friends could hear the bell, but as years passed, it fell silent for all of them. Even Sarah found one Christmas that she could no longer hear its sweet sound. Though I've grown old, the bell still rings for me, as it does for all who truly believe.
"""""]
query_LDA = map(queryLDA, query_texts)

In [257]:
query_LDA[0]

[(121, 0.47889469848744604),
 (268, 0.010369043214327713),
 (300, 0.032843703158862404),
 (372, 0.36411299654749596),
 (388, 0.03397801630032888),
 (442, 0.072386669349773364)]

In [258]:
simsLDA = indexLda[query_LDA[0]]

In [259]:
sorted(zip(text_names,simsLDA),key=lambda x: x[1],reverse=True)

[('I and My Chimney', 0.84503436),
 ('Moby Dick', 0.82158995),
 ('The Innocents Abroad', 0.7966786),
 ('A Tale of Two Cities', 0.74897534),
 ('Great Expectations', 0.65644503),
 ('Sylvie and Bruno', 0.63248408),
 ("A Connecticut Yankee in King Arthur's Court", 0.54631436),
 ('Northanger Abbey', 0.53531402),
 ('Emma', 0.52974641),
 ('Mansfield Park', 0.48785779),
 ('Hard Times', 0.4027071),
 ('Through the Looking-Glass', 0.34051466),
 ('Pride and Prejudice', 0.33996215),
 ('The Prince and the Pauper', 0.28635228),
 ('Adventures of Huckleberry Finn', 0.076895244),
 ('Grimms\xe2\x80\x99 Fairy Tales', 0.062974244),
 ('Alice in Wonderland', 0.053795528),
 ('Persuasion', 0.0),
 ("Eve's Diary", 0.0),
 ('A Tangled Tale', 0.0),
 ('Feeding the Mind', 0.0),
 ('The Confidence-Man: His Masquerade', 0.0),
 ('Pierre; or The Ambiguities', 0.0),
 ('Battle-Pieces and Aspects of the War', 0.0),
 ('A Christmas Carol', 0.0),
 ('Bleak House', 0.0)]

In [260]:
#results should've returned Christmas Carol ...

# Query

In [261]:
def query(doc):
    vec_bow = dictionary.doc2bow(preprocess_string(doc))
    vec_lsi = lsi[vec_bow] 
    return vec_lsi


In [262]:
query_texts = ["""
A young boy, who used to adore Christmas, hears a train whistle roar. To his astonishment, he finds the train is waiting for him. He sees a conductor who then proceeds to look up at his window. He runs downstairs and goes outside. The conductor explains the train is called the Polar Express, and is journeying to the North Pole. The boy then boards the train, which is filled with chocolate and candy, as well as many other children in their pajamas.

As the train reaches the North Pole, the boy and the other children see thousands of Christmas elves gathered at the center of town waiting to send Santa Claus on his way. The boy is handpicked by Santa to receive the first gift of Christmas. Realizing that he could choose anything in the world, the boy asks for one bell from one of the reindeer's harnesses. The boy places the bell in the pocket of his robe and all the children watch as Santa takes off into the night for his annual deliveries.

Later, on the train ride home, the boy discovers that the bell has fallen through a hole in his pocket. The boy arrives home and goes to his bedroom as the train pulls away. On Christmas morning, his sister finds a small package for the boy under the tree, behind all of the other gifts. The boy opens the box and discovers that it is the bell, delivered by Santa who found it on the seat of his sleigh. When the boy rings the bell, both he and his sister marvel at the beautiful sound. His parents, however, are unable to hear the bell and remark that it must be broken. The book ends with the following line:

At one time, most of my friends could hear the bell, but as years passed, it fell silent for all of them. Even Sarah found one Christmas that she could no longer hear its sweet sound. Though I've grown old, the bell still rings for me, as it does for all who truly believe.
"""""]
query_lsis = map(query,query_texts)

In [263]:
query_lsis[0]

[(0, 3.2531027844569107),
 (1, -0.47481108844777581),
 (2, -0.47895996781171501),
 (3, -0.13601670896506124),
 (4, -0.9602897258367038),
 (5, -0.18758601141049552),
 (6, -0.61256617776703315),
 (7, 0.36834101284291332),
 (8, 0.48536178641945477),
 (9, 0.49341574325114707),
 (10, -0.87172071000188911),
 (11, 0.9008261314373962),
 (12, 0.89783688906093762),
 (13, 0.28626865963913234),
 (14, -0.67682059228664293),
 (15, -0.31499504702696413),
 (16, -0.011501488957817533),
 (17, 0.2169213193781456)]

In [264]:
sims = index[query_lsis[0]]

In [265]:
print(list(enumerate(sims)))

[(0, 0.56123447), (1, 0.47847137), (2, 0.57781523), (3, 0.58258545), (4, 0.55238688), (5, 0.56871933), (6, 0.80295658), (7, 0.75257635), (8, 0.82462591), (9, 0.47863063), (10, 0.3054468), (11, 0.32612211), (12, 0.58258545), (13, 0.61943078), (14, 0.58258545), (15, 0.57197732), (16, 0.58258545), (17, 0.58258545), (18, 0.58258545), (19, 0.40270275), (20, 0.67318493), (21, 0.58258545), (22, 0.63998216), (23, 0.58258545), (24, 0.78529811), (25, 0.63453817)]


In [266]:
sorted(zip(text_names,sims),key=lambda x: x[1],reverse=True)

[('The Innocents Abroad', 0.82462591),
 ('The Prince and the Pauper', 0.80295658),
 ('Great Expectations', 0.78529811),
 ("A Connecticut Yankee in King Arthur's Court", 0.75257635),
 ('A Tale of Two Cities', 0.67318493),
 ('Hard Times', 0.63998216),
 ('Grimms\xe2\x80\x99 Fairy Tales', 0.63453817),
 ('Sylvie and Bruno', 0.61943078),
 ('Persuasion', 0.58258545),
 ('A Tangled Tale', 0.58258545),
 ('Feeding the Mind', 0.58258545),
 ('The Confidence-Man: His Masquerade', 0.58258545),
 ('Pierre; or The Ambiguities', 0.58258545),
 ('Battle-Pieces and Aspects of the War', 0.58258545),
 ('A Christmas Carol', 0.58258545),
 ('Bleak House', 0.58258545),
 ('Northanger Abbey', 0.57781523),
 ('Moby Dick', 0.57197732),
 ('Adventures of Huckleberry Finn', 0.56871933),
 ('Pride and Prejudice', 0.56123447),
 ('Mansfield Park', 0.55238688),
 ("Eve's Diary", 0.47863063),
 ('Emma', 0.47847137),
 ('I and My Chimney', 0.40270275),
 ('Through the Looking-Glass', 0.32612211),
 ('Alice in Wonderland', 0.3054468)